In [16]:


#pdf plumber
#csv.preview
import pandas as pd
#import unicode character database
import unicodedata
#import regular expression operations
import re

#import natural language toolkit
import nltk


#import our aquire
from acquire import *


#import our stopwords list
from nltk.corpus import stopwords
from copy import deepcopy








def split_apply_join(funct,listobj):
    'helperfuction letters'

    mapped=map(funct, listobj)
    mapped=list(mapped)
    mapped=''.join(mapped)
  
    return mapped






def stopfilter(text,stop_words_extend_reduce=["'"]):
    'we use symmetric difference so if a is already in stop words then it will be added to our third set else our third set will be missing it'
    #create oujr english stopwords list

    stops = set(stopwords.words('english'))
    stop_words_extend_reduce=set(stop_words_extend_reduce)
    stops=stops.symmetric_difference(stop_words_extend_reduce)

    # stops=(stops|stop_words_extend)-exclude_words
    #another way
    
    filtered=list(filter((lambda x: x not in stops), text.split()))
    filtered=' '.join(filtered)

    return filtered









def basic_clean(text,regexfilter=r'[^a-z0-9\'\s]'):
    '''   
    Filters out all special characters if you need to edit then supply a new regex filter 
    
    
    
    
    '''
    #make a copy and begin to transform it
    newtext = text.lower()

    #encode into ascii then decode
    newtext = unicodedata.normalize('NFKD', newtext)\
    .encode('ascii', 'ignore')\
    .decode('utf-8')

    #use re.sub to remove special characters
    newtext = re.sub(fr'{regexfilter}', '', newtext)

    return newtext

 


def tokenizer(text,regexfilter=r'[^a-z0-9\'\s]'):
    ''' 
    For a large file just save it locally
    
    
    
    
    
    '''
    newtext=basic_clean(text,regexfilter=regexfilter)
    #make ready tokenizer object
    tokenize = nltk.tokenize.ToktokTokenizer()
    #use the tokenizer
    newtext = tokenize.tokenize(newtext, return_str=True)
    return newtext





    
    
    
    


def stemmed(text,regexfilter=r'[^a-z0-9\'\s]'):
    '''    
    Takes text, tokenizes it, stems it
    
    
    
    
    
    '''
    #make ready porter stemmer object
    newtext=tokenizer(text,regexfilter=regexfilter)
    ps = nltk.porter.PorterStemmer()
    stemmedlist=split_apply_join(ps.stem,newtext)
    return stemmedlist




def lemmatizor(text,regexfilter=r'[^a-z0-9\'\s]'):
    '''    
    
      Takes text, tokenizes it, lemmatizes it
    
    
    
    
    
    '''

    #make ready the lemmatizer object
    newtext=tokenizer(text,regexfilter=regexfilter)
    wnl = nltk.stem.WordNetLemmatizer()
    lemmatized=split_apply_join(wnl.lemmatize,newtext)
    return lemmatized




def dictlist_super_NLP_comp(dictlist,regexfilter=r'[^a-z0-9\'\s]',stop_words_extend_reduce=["'"]):
    ''
    ndictlist=deepcopy(dictlist)
    mapper=[]
    interestingkeys=list(ndictlist.keys())
    for i in range(0,len(ndictlist)):           
            k=interestingkeys[i]
            text=ndictlist.get(k)         
            org={f'org':text}
            clean=basic_clean(text,regexfilter=regexfilter)
            cleaned=({f'cleaned':clean})
            lemmatized=lemmatizor(text,regexfilter)
            stopfilteredlemitezed=stopfilter(lemmatized,stop_words_extend_reduce=stop_words_extend_reduce)
            lemma={f'lemmatized':stopfilteredlemitezed}
            stem=stemmed(text,regexfilter)
            stopfilteredstem=stopfilter(stem,stop_words_extend_reduce=stop_words_extend_reduce)
            stemma={f'stemmed':stopfilteredstem}
            mapper.append({k:dict(**org,**cleaned,**lemma,**stemma)})

    
 
                
               


  
       
    return mapper


In [17]:
datadf=pd.read_json('data.json')

len(datadf)

# df_C=datadf[datadf['language']=='C'];df_C.head(n=30)


# df_Python=datadf[datadf['language']=='Python'];df_Python.head(n=30)


# df_Java=datadf[datadf['language']=='Java'];df_Java.head(n=30)





all_articles=(datadf['readme_contents']).to_dict()



In [18]:
newdf=dictlist_super_NLP_comp(all_articles)


In [19]:

a=pd.DataFrame(newdf[0]).T
for i in range(1,len(newdf)):
    b=pd.DataFrame(newdf[i]).T
    a=pd.merge(a,b,how='outer',right_index=False,left_index=False)


    


In [20]:
df=pd.merge(datadf,a,how='left',right_index=True,left_index=True)
display(df.head(),datadf.head())## Verify merge
datadf=df #finish prepped


,repo,language,readme_contents,cleaned,lemmatized,org,stemmed
0,cstack/db_tutorial,C,# Let's Build a Simple Database\n\n[View rende...,let's build a simple database\n\nview rendere...,let build simple database view rendered tutori...,# Let's Build a Simple Database\n\n[View rende...,let build simple database view rendered tutori...
1,rui314/chibicc,C,# chibicc: A Small C Compiler\n\n(The old mast...,chibicc a small c compiler\n\nthe old master ...,chibicc small c compiler old master moved hist...,# chibicc: A Small C Compiler\n\n(The old mast...,chibicc small c compiler old master moved hist...
2,nelhage/reptyr,C,"reptyr - A tool for ""re-ptying"" programs.\n===...",reptyr a tool for reptying programs\n\n\nrept...,reptyr tool reptying programs reptyr utility t...,"reptyr - A tool for ""re-ptying"" programs.\n===...",reptyr tool reptying programs reptyr utility t...
3,EyalAr/lwip,C,[![Version](http://img.shields.io/npm/v/lwip.s...,versionhttpimgshieldsionpmvlwipsvghttpswwwnpmj...,versionhttpimgshieldsionpmvlwipsvghttpswwwnpmj...,[![Version](http://img.shields.io/npm/v/lwip.s...,versionhttpimgshieldsionpmvlwipsvghttpswwwnpmj...
4,ibireme/yyjson,C,\n# Introduction\n\n[![Build](https://flat.bad...,\n introduction\n\nbuildhttpsflatbadgennetgith...,introduction buildhttpsflatbadgennetgithubstat...,\n# Introduction\n\n[![Build](https://flat.bad...,introduction buildhttpsflatbadgennetgithubstat...


,repo,language,readme_contents
0,cstack/db_tutorial,C,# Let's Build a Simple Database\n\n[View rende...
1,rui314/chibicc,C,# chibicc: A Small C Compiler\n\n(The old mast...
2,nelhage/reptyr,C,"reptyr - A tool for ""re-ptying"" programs.\n===..."
3,EyalAr/lwip,C,[![Version](http://img.shields.io/npm/v/lwip.s...
4,ibireme/yyjson,C,\n# Introduction\n\n[![Build](https://flat.bad...


In [21]:

reorder=['language',
'repo',
 'readme_contents',
 'cleaned',
 'stemmed',
 'lemmatized']
  
datadf=datadf[reorder]



In [22]:

datadf




,language,repo,readme_contents,cleaned,stemmed,lemmatized
0,C,cstack/db_tutorial,# Let's Build a Simple Database\n\n[View rende...,let's build a simple database\n\nview rendere...,let build simple database view rendered tutori...,let build simple database view rendered tutori...
1,C,rui314/chibicc,# chibicc: A Small C Compiler\n\n(The old mast...,chibicc a small c compiler\n\nthe old master ...,chibicc small c compiler old master moved hist...,chibicc small c compiler old master moved hist...
2,C,nelhage/reptyr,"reptyr - A tool for ""re-ptying"" programs.\n===...",reptyr a tool for reptying programs\n\n\nrept...,reptyr tool reptying programs reptyr utility t...,reptyr tool reptying programs reptyr utility t...
3,C,EyalAr/lwip,[![Version](http://img.shields.io/npm/v/lwip.s...,versionhttpimgshieldsionpmvlwipsvghttpswwwnpmj...,versionhttpimgshieldsionpmvlwipsvghttpswwwnpmj...,versionhttpimgshieldsionpmvlwipsvghttpswwwnpmj...
4,C,ibireme/yyjson,\n# Introduction\n\n[![Build](https://flat.bad...,\n introduction\n\nbuildhttpsflatbadgennetgith...,introduction buildhttpsflatbadgennetgithubstat...,introduction buildhttpsflatbadgennetgithubstat...
...,...,...,...,...,...,...
265,Java,Amterson/AlginProject,\n### 1.非中文单词不够一行会自动截断，用符号“-”连接起来；\n\n### 2.适配...,\n 1\n\n 2textviewandroidgravityandroidtextali...,1 2textviewandroidgravityandroidtextalignmentg...,1 2textviewandroidgravityandroidtextalignmentg...
266,Java,twitch4j/twitch4j,"<p align=""center""><a href=""https://twitch4j.gi...",p aligncentera hrefhttpstwitch4jgithubioimg sr...,p aligncentera hrefhttpstwitch4jgithubioimg sr...,p aligncentera hrefhttpstwitch4jgithubioimg sr...
267,Java,ittianyu/POCenter,## 外包集中营 ##\n\n整合多个软件外包平台项目信息，替你筛选优质项目\n\n![MI...,\n\n\n\nmit licensehttpsimgshieldsiogithubli...,mit licensehttpsimgshieldsiogithublicensemasha...,mit licensehttpsimgshieldsiogithublicensemasha...
268,Java,acloudyh/springCloud,## springCloud 学习\n\n[springCloud-config配置中心](...,springcloud \n\nspringcloudconfighttpsgithubc...,springcloud springcloudconfighttpsgithubcomacl...,springcloud springcloudconfighttpsgithubcomacl...
